In [1]:
#Author:  Anthony Tugman
#Title:   Predicting Hotel Prices Using Linear Regression

#Import required libraries and dependencies

from google.colab import files
from scipy import stats
uploaded = files.upload()
import io
import pandas as pd
import numpy as np
import pandas

Saving hotel_bookings.csv to hotel_bookings (2).csv


In [2]:
#Load data for prep

df = pd.read_csv(io.BytesIO(uploaded['hotel_bookings.csv']))
pd.set_option('display.max_columns', None)

In [3]:
#Display original attributes of data

print("shape: ",df.shape)
print("duplicate: ",df.duplicated().sum())

shape:  (119390, 32)
duplicate:  31994


In [4]:
#Display data

df.head()

#Identify irrelavant categories (as described in report)
#meal, country, agent, company, babies, children,reservation_status_date, days_in_waiting_list, market_segment, distribution_channel

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [5]:
#Remove duplicates

df.drop_duplicates(inplace=True)

#Remove null categories and additional unnecessary categories

df.drop('company', axis=1, inplace=True)
df.drop('country', axis=1, inplace=True)
df.drop('agent', axis =1, inplace=True)
df.drop('babies', axis =1, inplace=True)
df.drop('reservation_status_date', axis =1, inplace=True)
df.drop('days_in_waiting_list', axis=1, inplace=True) 
df.drop('children', axis =1, inplace=True)
df.drop('meal', axis =1, inplace=True)
df.drop('market_segment', axis =1, inplace=True)
df.drop('distribution_channel', axis =1, inplace=True)

df.isnull().sum()

hotel                             0
is_canceled                       0
lead_time                         0
arrival_date_year                 0
arrival_date_month                0
arrival_date_week_number          0
arrival_date_day_of_month         0
stays_in_weekend_nights           0
stays_in_week_nights              0
adults                            0
is_repeated_guest                 0
previous_cancellations            0
previous_bookings_not_canceled    0
reserved_room_type                0
assigned_room_type                0
booking_changes                   0
deposit_type                      0
customer_type                     0
adr                               0
required_car_parking_spaces       0
total_of_special_requests         0
reservation_status                0
dtype: int64

In [6]:
#determine if customer was given requested room by comparing
#reserved_room_type to assigned_room_type creating a new column
#called room_correct where 1 indicates the desired room was given and 0
#indicates that it was not

df['room_correct']= (df['reserved_room_type']==df['assigned_room_type'])
del df['reserved_room_type']
del df['assigned_room_type']

#Convert Strings to Numerical Values
df = df.replace(['City Hotel', 'False', 'Refundable', 'Non Refund', 'No-Show', 
                 'Transient'],'0')
df = df.replace(['Resort Hotel', 'True', 'January', 'Canceled', 
                 'Transient-Party'], '1')
df = df.replace(['February', 'No Deposit', 'Check-Out', 'Contract'], '2')
df = df.replace(['March', 'Group'], '3')
df = df.replace(['April'], '4')
df = df.replace(['May'], '5')
df = df.replace(['June'], '6')
df = df.replace(['July'], '7')
df = df.replace(['August'], '8')
df = df.replace(['September'], '9')
df = df.replace(['October'], '10')
df = df.replace(['November'], '11')
df = df.replace(['December'], '12')
df['room_correct'] = df['room_correct'].astype(int)

df.drop_duplicates(inplace=True)

#Display updated attributes of data

print("shape: ",df.shape)
print("duplicate: ",df.duplicated().sum())

df.head()

shape:  (84851, 21)
duplicate:  0


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,room_correct
0,1,0,342,2015,7,27,1,0,0,2,0,0,0,3,2,0,0.0,0,0,2,1
1,1,0,737,2015,7,27,1,0,0,2,0,0,0,4,2,0,0.0,0,0,2,1
2,1,0,7,2015,7,27,1,0,1,1,0,0,0,0,2,0,75.0,0,0,2,0
3,1,0,13,2015,7,27,1,0,1,1,0,0,0,0,2,0,75.0,0,0,2,1
4,1,0,14,2015,7,27,1,0,2,2,0,0,0,0,2,0,98.0,0,1,2,1
